In [2]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import math

from shared.utils import formalize_data
from shared.random_rules import RandomRules


In [3]:
a = formalize_data("09 12 27 33 24 30")
print(a)

('091224273033', (9, 12, 24, 27, 30, 33))


In [4]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)



In [5]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')





In [6]:
lst = []
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int = formalize_data(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  
  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  left_scores = [round(b, 4) for b in rr.calculate_left_scores(bolillas_int)]
  right_scores = [round(b, 4) for b in rr.calculate_right_scores(bolillas_int)]
  lr_scores = [round(left + right,4) for left, right in zip(left_scores, right_scores)]
  row_score = round(sum(lr_scores),4)

  shared_values = list(set(left_scores[0:5]) & set(right_scores[1:6]))
  shared_values = [round(s, 4) for s in shared_values]
  cols.extend([score for score in left_scores])
  cols.extend([score for score in right_scores])
  # cols.extend(lr_scores)
  cols.append(shared_values)
  cols.append(len(shared_values))
  cols.append(row_score)
  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

with open('./gnche.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'is_rare_combination', 'reason',
    "score_left_b1", "score_left_b2", "score_left_b3", "score_left_b4", "score_left_b5",
    "score_r8_b1", "score_r8_b2", "score_r8_b3", "score_r8_b4", "score_r8_b5",
    # "score_lr_b1", "score_lr_b2", "score_lr_b3", "score_lr_b4", "score_lr_b5", "score_lr_b6",
    "shared_values", "shared_values_count",
    "score_row",
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'shared_values_count': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))


TOTAL_LINES=7709


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,shared_values,shared_values_count,score_row,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,bool,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,str,i64
"""24/03/2025""","""4165""","""32 29 27 21 06""","""0621272932""","[6, 21, … 32]",6,21,27,29,32,false,0,0.05,0.0385,0.0833,0.1429,0.1667,0.05,0.04,0.0385,0.0357,0.1667,"[0.0385, 0.1667]",2,0.8123,"""15_06_02_03""",0
"""23/03/2025""","""4164""","""13 10 04 15 08""","""0408101315""","[4, 8, … 15]",4,8,10,13,15,true,16,0.1429,0.0357,0.04,0.0417,0.0455,0.1429,0.125,0.1,0.0909,0.0455,[0.0455],1,0.8101,"""04_02_03_02""",-5
"""22/03/2025""","""4163""","""19 32 17 10 22""","""1017192232""","[10, 17, … 32]",10,17,19,22,32,false,0,0.0625,0.0455,0.0625,0.0667,0.0769,0.0625,0.0588,0.0526,0.0357,0.0769,[0.0769],1,0.6006,"""07_02_03_10""",0
"""21/03/2025""","""4162""","""32 33 01 17 26""","""0117263233""","[1, 17, … 33]",1,17,26,32,33,false,0,0.0625,0.0323,0.0625,0.125,0.3333,0.0625,0.0417,0.0345,0.0345,0.3333,[0.3333],1,1.1221,"""16_09_06_01""",-6
"""20/03/2025""","""4161""","""32 24 17 12 27""","""1217242732""","[12, 17, … 32]",12,17,24,27,32,false,0,0.0625,0.05,0.0625,0.1,0.125,0.0625,0.0455,0.0417,0.0357,0.125,[0.125],1,0.7104,"""05_07_03_05""",12
"""19/03/2025""","""4160""","""01 13 07 22 14""","""0107131422""","[1, 7, … 22]",1,7,13,14,22,true,8,0.1667,0.0323,0.0385,0.0476,0.0476,0.1667,0.0909,0.0909,0.0556,0.0476,[0.0476],1,0.7844,"""06_06_01_08""",17
"""18/03/2025""","""4159""","""07 34 16 12 21""","""0712162134""","[7, 12, … 34]",7,12,16,21,34,false,0,0.0909,0.04,0.0476,0.0556,0.0714,0.0909,0.0714,0.0556,0.0333,0.0714,"[0.0714, 0.0556]",2,0.6281,"""05_04_05_13""",5
"""17/03/2025""","""4158""","""20 30 04 08 24""","""0408202430""","[4, 8, … 30]",4,8,20,24,30,true,20,0.1429,0.0357,0.04,0.0714,0.0909,0.1429,0.0556,0.0476,0.0385,0.0909,[0.0909],1,0.7564,"""04_12_04_06""",26
"""16/03/2025""","""4157""","""26 10 21 16 30""","""1016212630""","[10, 16, … 30]",10,16,21,26,30,true,8,0.0667,0.0455,0.0588,0.0769,0.1111,0.0667,0.0526,0.0435,0.0385,0.1111,[0.1111],1,0.6714,"""06_05_05_04""",-2


# ANALYZE RANDOM!
You can't analyze random bc it's random! But you can find "order" in "entropy" I guess. Like when you have your room messed up but somehow you can find everything. Nevermid.
Just look at the results below

In [7]:
unijumps = dfRows.group_by(
  'unijump'
).agg(
  pl.count('unijump').alias('count')
).sort(
  "count", descending=True
)


In [8]:
unijumps.write_excel("unijump_gn.xlsx")

In [9]:
import json
percent_one = math.ceil(TOTAL_LINES * 0.01)
print(f'{percent_one=}')
print(unijumps[0:percent_one]['unijump'].to_list())
lst_unijumps = unijumps[0:percent_one]['unijump'].to_list()

with open('unijump_gn.json', 'w') as f:
  f.write(json.dumps(lst_unijumps))
  

percent_one=78
[-1, 7, -2, -3, 3, 0, -4, 4, 5, 2, 6, -5, 10, -12, -6, 1, -9, -7, -11, 11, 9, 12, 8, -8, -10, -13, 14, -15, -14, 16, 13, 15, 19, -16, 21, -17, 17, 18, -18, -20, -19, -21, 23, -22, -23, 22, 20, 25, -24, 26, 24, 27, -28, -26, 29, -27, -25, -30, -33, -29, -32, 28, 35, 34, 31, 30, 36, 33, 32, -35, -38, -31, -34, 39, 40, 37, -39, -36]


In [10]:
dfRows.group_by(
  'reason'
).agg(
  pl.count('reason').alias('count')
).sort(
  "count", descending=True
)


reason,count
u8,u32
0,3787
16,1932
8,936
24,554
1,98
…,…
18,19
11,13
19,5


In [11]:
dfRows.group_by(
  'shared_values_count'
).agg(pl.count('shared_values_count').alias('count'))

shared_values_count,count
u8,u32
3,286
4,3
1,4795
2,2625


In [12]:
df_jumps_map = dfRows.group_by(
  'jumps_map'
).agg(pl.count('jumps_map').alias('count')).sort("count", descending=True)

display(df_jumps_map[0:10])

repeated_jumps_map = df_jumps_map.filter(pl.col("count") > 1)["jumps_map"].to_list()
print(f'MORE THAN 1 {len(repeated_jumps_map)=}')


q = dfRows.filter(
  pl.col("jumps_map").is_in(repeated_jumps_map) 
).sort("jumps_map")

display(q)

with open ("./jumps_map_gn.json", "w") as file:
  json.dump(df_jumps_map["jumps_map"].to_list(), file)



jumps_map,count
str,u32
"""03_06_02_03""",4
"""02_06_05_01""",4
"""01_03_02_09""",4
"""02_02_03_03""",4
"""01_01_09_07""",4
"""01_06_14_02""",4
"""01_05_02_03""",4
"""03_02_01_04""",4
"""03_06_02_01""",4


MORE THAN 1 len(repeated_jumps_map)=858


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,shared_values,shared_values_count,score_row,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,bool,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,str,i64
"""02/06/2010""","""2431""","""04 05 06 07 13""","""0405060713""","[4, 5, … 13]",4,5,6,7,13,true,24,0.25,0.0357,0.0357,0.0357,0.0357,0.25,0.25,0.25,0.1111,0.0357,"[0.25, 0.0357]",2,1.2896,"""01_01_01_06""",5
"""14/12/2003""","""68""","""22 23 25 24 31""","""2223242531""","[22, 23, … 31]",22,23,24,25,31,true,8,0.0455,0.1,0.1,0.1,0.1,0.0455,0.0455,0.0455,0.037,0.1,"[0.0455, 0.1]",2,0.719,"""01_01_01_06""",5
"""21/05/2022""","""3127""","""26 19 22 18 20""","""1819202226""","[18, 19, … 26]",18,19,20,22,26,true,24,0.0556,0.0714,0.0714,0.0714,0.0769,0.0556,0.0556,0.0526,0.0455,0.0769,"[0.0556, 0.0769]",2,0.6329,"""01_01_02_04""",0
"""09/03/2012""","""3077""","""01 02 03 05 09""","""0102030509""","[1, 2, … 9]",1,2,3,5,9,true,8,1.0,0.0323,0.0323,0.0323,0.0333,1.0,1.0,0.5,0.2,0.0333,"[0.0333, 1.0]",2,3.8635,"""01_01_02_04""",0
"""17/05/2007""","""1318""","""12 14 16 20 13""","""1213141620""","[12, 13, … 20]",12,13,14,16,20,true,8,0.0833,0.05,0.05,0.05,0.0526,0.0833,0.0833,0.0769,0.0625,0.0526,"[0.0833, 0.0526]",2,0.6445,"""01_01_02_04""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""19/12/2007""","""1534""","""32 30 29 25 02""","""0225293032""","[2, 25, … 32]",2,25,29,30,32,true,16,0.0417,0.0333,0.125,0.2,0.2,0.0417,0.037,0.037,0.0357,0.2,[0.2],1,0.9514,"""23_04_01_02""",-12
"""29/01/2015""","""0535""","""03 26 30 32 33""","""0326303233""","[3, 26, … 33]",3,26,30,32,33,true,16,0.04,0.0345,0.1429,0.25,0.3333,0.04,0.0357,0.0345,0.0345,0.3333,"[0.0345, 0.3333]",2,1.2787,"""23_04_02_01""",-16
"""22/01/2013""","""3396""","""03 26 30 32 33""","""0326303233""","[3, 26, … 33]",3,26,30,32,33,true,16,0.04,0.0345,0.1429,0.25,0.3333,0.04,0.0357,0.0345,0.0345,0.3333,"[0.0345, 0.3333]",2,1.2787,"""23_04_02_01""",-16


In [13]:
def show_b_stats (col: str):
  df_viz = dfRows.group_by(
    col
  ).agg(pl.count(col).alias('count'))

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)



In [14]:
show_b_stats('score_left_b1')
show_b_stats('score_r8_b1')


score_left_b1,count
f64,u32
0.0476,162
0.0769,400
0.0333,3
0.125,478
0.0588,294
…,…
0.0909,501
0.0385,42
0.0417,80


alt.Chart(...)

score_r8_b1,count
f64,u32
0.0769,400
0.0476,162
0.0588,294
0.125,478
0.0333,3
…,…
0.1667,475
0.0909,501
0.0526,199


alt.Chart(...)

In [15]:
show_b_stats('score_left_b2')
show_b_stats('score_r8_b2')

score_left_b2,count
f64,u32
0.0333,971
0.0588,109
0.125,8
0.0476,290
0.0769,43
…,…
0.0385,572
0.0909,24
0.0417,371


alt.Chart(...)

score_r8_b2,count
f64,u32
0.125,217
0.0476,380
0.0588,407
0.0333,35
0.0769,420
…,…
0.1667,180
0.0909,389
0.0417,252


alt.Chart(...)

In [16]:
show_b_stats('score_left_b3')
show_b_stats('score_r8_b3')

score_left_b3,count
f64,u32
0.0476,501
0.0769,199
0.0333,237
0.0588,328
0.125,80
…,…
0.0385,475
0.1667,42
0.0526,400


alt.Chart(...)

score_r8_b3,count
f64,u32
0.0333,224
0.125,72
0.0476,476
0.0769,185
0.0588,331
…,…
0.0909,159
0.1667,32
0.0526,428


alt.Chart(...)

In [17]:
show_b_stats('score_left_b4')
show_b_stats('score_r8_b4')

score_left_b4,count
f64,u32
0.0333,38
0.0476,389
0.0769,393
0.125,252
0.0588,424
…,…
0.0909,380
0.1667,203
0.0556,421


alt.Chart(...)

score_r8_b4,count
f64,u32
0.0333,892
0.0769,36
0.0588,106
0.125,11
0.0476,238
…,…
0.0909,19
0.1667,5
0.0556,156


alt.Chart(...)

In [18]:
show_b_stats('score_left_b5')
show_b_stats('score_r8_b5')

score_left_b5,count
f64,u32
0.0769,428
0.0333,4
0.0476,159
0.125,487
0.0588,270
…,…
0.1667,458
0.0385,32
0.0556,246


alt.Chart(...)

score_r8_b5,count
f64,u32
0.0333,4
0.0588,270
0.125,487
0.0769,428
0.0476,159
…,…
0.1667,458
0.0909,476
0.0526,185


alt.Chart(...)